In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D


In [ ]:
def distance(q1,q2):
    return(round(np.linalg.norm(q2-q1),2))

In [ ]:
def u_att(zeta,d_star,q,q0):
    d=distance(q,q0)
    if (d<=d_star):
        return(((zeta * (d**2) * 1/2)))
    else:
        return(((d_star * zeta * d)- (1/2 * zeta * (d_star)**2)))

def att_grad(zeta,d_star,q,q0):
    d=distance(q,q0)
    if (d<=d_star):
        return((zeta * (q-q0) ))
    else:
        return((d_star * zeta * (q-q0)/d))

In [ ]:
def u_rep(n,q_star,q,oq):
    r= []
    for array in (oq.T):
        array=array.reshape(2,1)
        d= distance(q,array)
        if d<=q_star:
            r.append((0.5 * n * ((1/d)-(1/q_star))**2))
        else:
            r.append(0)
    return(sum(r))


    
def rep_grad(n,q_star,q,oq):
    r=np.zeros((2,1))
    for array in (oq.T):
        array=array.reshape(2,1)
        d= distance(q,array)
        if d<=q_star:
            r=np.append(r,(n* ((1/q_star)-(1/d))* ((q-(array))/(d**3))),axis=1)
        else:
            r=np.append(r,np.zeros((2,1)),axis= 1)

    return(np.sum(r,axis=1).reshape(2,1))



In [ ]:
def GradientDescent(q,oq,q0,alpha,max_iter,n,q_star,zeta,d_star,U_star):
    success=False
    U = u_rep(n,q_star,q,oq) + u_att(zeta,d_star,q,q0)
    U_hist=[U]
    q_hist= q
    for i in range(max_iter):
        
        if U > U_star:
            grad_total= rep_grad(n,q_star,q,oq) + att_grad(zeta,d_star,q,q0)
            q = q - alpha * (grad_total/np.linalg.norm(grad_total))
            U =  u_rep(n,q_star,q,oq) + u_att(zeta,d_star,q,q0)
            q_hist= np.hstack((q_hist,q))
            U_hist.append(U)
            if i % 25 == 0:
                print(f"Potential after {i} iterations is "+str(U))
                # print(q)
        else:
            print("Algorithm converged successfully and Robot has reached goal location")
            break
        if (i == max_iter-1):
            print("Robot is either at local minima or loop ran out of maximum  iterations")
            
    return(q_hist,U_hist)

In [ ]:
q=np.array(([0,0])).reshape(2,1)
q0=np.array([10,15]).reshape(2,1)
oq= np.array([[2.5,3],[4,6],[5,9],[7,11],[5,12],[5,11],[6,10],[2.5,5],[3,8]]).T
max_iter=1500
alpha=0.1
n=1
zeta= 1
U_star=0.1
q_star=3
d_star=1
q_hist, U_hist= GradientDescent(q,oq,q0,alpha,max_iter,n,q_star,zeta,d_star,U_star)


In [ ]:
fig1=plt.figure()
X= np.arange(1,len(U_hist)+1)
plt.plot(X,U_hist,color="blue")
plt.title("Potential Function Value vs Iterations")
plt.xlabel("Iterations")
plt.ylabel("Potential value")
plt.grid()
plt.show(fig1)

In [ ]:
%matplotlib widget
fig2 = plt.figure()
ax = Axes3D(fig2)

x= np.arange(0,max(q0)+5,alpha)
y= np.arange(0,max(q0)+5,alpha)
Z1 = np.empty((len(x),len(y)))
for j in range(len(y)):
    for i in range(len(x)):
        Z1[i,j]=u_rep(n,q_star,np.array([x[i],y[j]]).reshape(2,1),oq) + u_att(zeta,d_star,np.array([x[i],y[j]]).reshape(2,1),q0)

X1,Y1= np.meshgrid(x,y)

plot_potential = ax.plot_surface(X1, Y1, Z1,rstride=1, cstride=1,cmap='viridis', edgecolor='none')
ax.set_title("3D plot for potential value")
ax.set_xlabel('x-Coordinate')
ax.set_ylabel('y-Coordinate')
ax.set_zlabel('Potential')
plt.show(fig2)



In [ ]:
fig3, axes = plt.subplots()
for ob in (oq.T):
    ob=ob.reshape(1,2)
    plt.plot(ob[0,0],ob[0,1],marker="o")
    circle= plt.Circle((ob[0,0],ob[0,1]),1,fill=False)
    axes.set_aspect(1)
    axes.add_artist(circle)


for point in (q_hist.T):
    point=point.reshape(1,2)
    plt.plot(point[0,0],point[0,1],marker='.')

plt.title("Path followed by the Robot")
plt.xlabel("X")
plt.ylabel("Y")
plt.grid()
plt.show(fig3)